# Building a RAG System with SQuAD v2 Dataset

In this assignment, we are building a mini Retrieval-Augmented Generation (RAG) pipeline to answer questions using the SQuAD v2 dataset from Hugging Face.


In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_v2")
print("Dataset loaded successfully")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 11873/11873 [00:00<00:00, 1528091.42 examples/s]

Dataset loaded successfully


In [5]:
print(dataset["train"][0])
print(f"Training samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}
Training samples: 130319
Validation samples: 11873


## Preparing the Knowledge Base

We are extracting contexts from the dataset to build our knowledge base for retrieval.


In [6]:
# Extracting unique contexts from validation set
contexts = []
seen_contexts = set()

for item in dataset["validation"]:
    context = item["context"]
    if context not in seen_contexts:
        contexts.append(context)
        seen_contexts.add(context)

print(f"Extracted {len(contexts)} unique contexts")


Extracted 1204 unique contexts


## Implementing Semantic Retriever

We are loading a sentence transformer model to embed our knowledge base and enable semantic search.


In [8]:
from sentence_transformers import SentenceTransformer

retriever = SentenceTransformer('all-MiniLM-L6-v2')
print("Retriever model loaded")

Retriever model loaded


In [9]:
# Encoding all contexts
context_embeddings = retriever.encode(contexts, convert_to_tensor=True, show_progress_bar=False)
print("Contexts encoded successfully")

Contexts encoded successfully


## Loading Lightweight Open LLM

We are loading a lightweight open-access language model for generating answers.

In [11]:
from transformers import pipeline
import warnings

warnings.filterwarnings("ignore")

# Using Google's Flan-T5 which is optimized for question answering
generator = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)
print("LLM loaded successfully")

Device set to use cpu


LLM loaded successfully


## Building the RAG Pipeline

We are creating a function that combines retrieval and generation to answer questions.

In [19]:
from sentence_transformers import util

def answer_question(question, top_k=3):
    # Encoding the question
    question_embedding = retriever.encode(question, convert_to_tensor=True)
    
    # Computing cosine similarity
    scores = util.pytorch_cos_sim(question_embedding, context_embeddings)
    
    # Retrieving top-k contexts
    top_indices = scores.topk(k=top_k).indices[0].tolist()
    retrieved_contexts = [contexts[idx] for idx in top_indices]
    
    # Combining contexts
    combined_context = "\n\n".join(retrieved_contexts)
    
    # Creating prompt for LLM
    prompt = f"Context: {combined_context}\n\nQuestion: {question}\nAnswer:"
    
    # Generating answer
    response = generator(prompt, max_new_tokens=100, do_sample=False)
    
    
    return {
        "question": question,
        "retrieved_contexts": retrieved_contexts,
        "answer": response[0]['generated_text']
    }

print("RAG pipeline ready")

RAG pipeline ready


## Evaluating the System

We are testing our RAG system with sample questions from the SQuAD v2 dataset.

In [22]:
# Testing with first sample question
test_question = dataset["validation"][0]["question"]
result = answer_question(test_question)

print(f"Question: {result['question']}")
print(f"Answer: {result['answer'].split('Answer:')[-1].strip()}")
print(f"Retrieved Contexts: {result['retrieved_contexts']}")

Question: In what country is Normandy located?
Answer: France
Retrieved Contexts: ['In the course of the 10th century, the initially destructive incursions of Norse war bands into the rivers of France evolved into more permanent encampments that included local women and personal property. The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was situated in the former Frankish kingdom of Neustria. The treaty offered Rollo and his men the French lands between the river Epte and the Atlantic coast in exchange for their protection against further Viking incursions. The area corresponded to the northern part of present-day Upper Normandy down to the river Seine, but the Duchy would eventually extend west beyond the Seine. The territory was roughly equivalent to the old province of Rouen, and reproduced the Roman administrative structure of Gallia Lugdunensi

In [25]:
# Testing with another sample question
test_question_2 = dataset["validation"][10]["question"]
result_2 = answer_question(test_question_2)

print(f"Question: {result_2['question']}")
print(f"Answer: {result_2['answer'].split('Answer:')[-1].strip()}")
print(f"Retrieved Contexts: {result_2['retrieved_contexts']}")

Question: Who ruled the duchy of Normandy
Answer: Richard I of Normandy
Retrieved Contexts: ['In the course of the 10th century, the initially destructive incursions of Norse war bands into the rivers of France evolved into more permanent encampments that included local women and personal property. The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was situated in the former Frankish kingdom of Neustria. The treaty offered Rollo and his men the French lands between the river Epte and the Atlantic coast in exchange for their protection against further Viking incursions. The area corresponded to the northern part of present-day Upper Normandy down to the river Seine, but the Duchy would eventually extend west beyond the Seine. The territory was roughly equivalent to the old province of Rouen, and reproduced the Roman administrative structure of Gallia 

In [26]:
# Testing with a third sample question
test_question_3 = dataset["validation"][25]["question"]
result_3 = answer_question(test_question_3)

print(f"Question: {result_3['question']}")
print(f"Answer: {result_3['answer'].split('Answer:')[-1].strip()}")
print(f"Retrieved Contexts: {result_3['retrieved_contexts']}")

Question: What treaty was established in the 9th century?
Answer: treaty of Saint-Clair-sur-Epte
Retrieved Contexts: ['The principal Treaties that form the European Union began with common rules for coal and steel, and then atomic energy, but more complete and formal institutions were established through the Treaty of Rome 1957 and the Maastricht Treaty 1992 (now: TFEU). Minor amendments were made during the 1960s and 1970s. Major amending treaties were signed to complete the development of a single, internal market in the Single European Act 1986, to further the development of a more social Europe in the Treaty of Amsterdam 1997, and to make minor amendments to the relative power of member states in the EU institutions in the Treaty of Nice 2001 and the Treaty of Lisbon 2007. Since its establishment, more member states have joined through a series of accession treaties, from the UK, Ireland, Denmark and Norway in 1972 (though Norway did not end up joining), Greece in 1979, Spain and P

## Qualitative Evaluation Summary

We have successfully built a mini-RAG pipeline that:
- Loads and explores the SQuAD v2 dataset from Hugging Face
- Implements a semantic retriever using sentence transformers
- Uses a lightweight open LLM (Llama 3.2 1B) to generate answers
- Retrieves relevant contexts and generates contextually appropriate answers

The system demonstrates the core functionality of Retrieval-Augmented Generation by combining semantic search with language model capabilities to answer questions based on retrieved knowledge.


In [29]:
# Comprehensive evaluation of the RAG system
import time
import numpy as np

print("=" * 80)
print("RAG SYSTEM EVALUATION")
print("=" * 80)

# Testing retrieval accuracy on multiple questions
test_indices = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
retrieval_results = []
response_times = []

print("\nTesting 10 sample questions...\n")

for idx in test_indices:
    question = dataset["validation"][idx]["question"]
    actual_context = dataset["validation"][idx]["context"]
    
    start_time = time.time()
    result = answer_question(question, top_k=3)
    elapsed_time = time.time() - start_time
    
    # Checking if correct context was retrieved
    found = any(actual_context in rc or rc in actual_context for rc in result['retrieved_contexts'])
    retrieval_results.append(found)
    response_times.append(elapsed_time)

# Calculating metrics
retrieval_accuracy = sum(retrieval_results) / len(retrieval_results) * 100
avg_response_time = np.mean(response_times)

print("RESULTS:")
print("-" * 80)
print(f"Retrieval Accuracy: {retrieval_accuracy:.1f}% ({sum(retrieval_results)}/{len(retrieval_results)} questions)")
print(f"Average Response Time: {avg_response_time:.2f} seconds")
print(f"Total Contexts in Knowledge Base: {len(contexts)}")
print("-" * 80)

print("\nQUALITATIVE ASSESSMENT:")
print(f"✓ Achieves {retrieval_accuracy:.1f}% retrieval accuracy on test samples")
print(f"✓ Maintains efficient response time (~{avg_response_time:.2f}s per query)")
print("=" * 80)

RAG SYSTEM EVALUATION

Testing 10 sample questions...

RESULTS:
--------------------------------------------------------------------------------
Retrieval Accuracy: 90.0% (9/10 questions)
Average Response Time: 3.03 seconds
Total Contexts in Knowledge Base: 1204
--------------------------------------------------------------------------------

QUALITATIVE ASSESSMENT:
✓ Achieves 90.0% retrieval accuracy on test samples
✓ Maintains efficient response time (~3.03s per query)
